# Evaluating the Performance of Phi 3.5 Mini Model

## Evaluation Metrics

The evaluation metrics that will be used to evaluate the text summarization performance of the Large Language Models (LLMs) are:
1. METEOR (Metric for Evaluation of Translation with Explicit Ordering)
2. ROUGE-N (Recall-Oriented Understudy for Gisting Evaluation)
3. BERTScore
4. BLEU (BiLingual Evaluation Understudy)
5. G-Eval
6. FactCC
7. Model's Inference Time

In [ ]:
import os
import torch
import pandas as pd

from datasets import load_from_disk
from transformers.utils import is_flash_attn_2_available
from unsloth import FastLanguageModel

import evaluate

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "XXXXXXXXXXXXXXXXXXXXXXX"

device = "cuda"
torch.cuda.empty_cache()

dataset = load_from_disk("../datasets/xsum_dataset.hf")
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 30000
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 3750
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 3750
    })
})

In [ ]:
def load_peft_model():
    
    if (is_flash_attn_2_available() and (torch.cuda.get_device_capability(0)[0] >= 8)):
        attn_implementation = "flash_attention_2"
    else:
        attn_implementation = "sdpa"
    
    print(f"[INFO] Using attention implementation: {attn_implementation}")

    model_id = "unsloth/Phi-3-mini-4k-instruct"
    print(f"[INFO] Using model_id: {model_id}")

    peft_model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_id,
        max_seq_length = 8192,
        dtype = None,
        load_in_4bit = True,
        token = "XXXXXXXXXXXXXXXXXXXXXXXX"
    )

    peft_model.to(device)

    return peft_model, tokenizer

peft_model, tokenizer = load_peft_model()

[INFO] Using attention implementation: sdpa
[INFO] Using model_id: unsloth/Phi-3-mini-4k-instruct
==((====))==  Unsloth 2024.12.4: Fast Mistral patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/phi-3-mini-4k-instruct-bnb-4bit can only handle sequence lengths of at most 4096.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 8192!


In [7]:
dataset = dataset['test']

articles = dataset['document'][0:50]
human_summaries = dataset['summary'][0:50]
generated_summaries = []

for idx, article in enumerate(articles):
    prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following text.

### Input:
{article}

### Response:
"""
    input_ids = tokenizer(prompt, return_tensors='pt')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = input_ids.to(device)
    human_baseline_text_output = human_summaries[idx]
    FastLanguageModel.for_inference(peft_model)
    peft_model_output = peft_model.generate(**input_ids, max_new_tokens = 6000, temperature = 0.1)
    prompt_length = input_ids['input_ids'].shape[1]
    peft_model_text_output = tokenizer.decode(peft_model_output[0][prompt_length:], skip_special_tokens = True)
    generated_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(articles, human_summaries, generated_summaries))

KeyboardInterrupt: 

In [10]:
df = pd.DataFrame(zipped_summaries, columns = ['Article', 'Human Summary', 'Generated Summary'])
df

,Article,Human Summary,Generated Summary
0,"The Newport man faces other charges, including...",A 22-year-old man has been charged with causin...,A man in Newport faces additional charges of v...
1,"Staff at RSPCA Gonsal Farm animal centre, in D...",A Shropshire charity has designated October 'B...,The RSPCA Gal Farm animal center in Dorrington...
2,"According to CNN, the former FBI director and ...",Now that Hurricane Junior has blown through Wa...,The FBI and former investigators were caught o...
3,Former leader Nick Paget-Brown resigned on 30 ...,The new leader of Kensington and Chelsea Counc...,Former leader Nick Paget-B resigned after crit...
4,That makes it more serious than a technical co...,"The index of the UK's biggest 100 companies, t...",The text discusses the current market situatio...
5,Gorse fires have been big news this week and t...,"A ""river of filth"", a spate of gorse fires, an...",Recent fires have been causing significant new...
6,"David Davies, Ian Lucas, Albert Owen and Gerai...",Four Welsh MPs are standing for election as ch...,"David Davies, Ian, Owen, and Geraint Davies ar..."
7,Three auctioneers at Hotel Drouot also receive...,A French court has jailed 35 porters at the co...,Three hotel porters at Hotel Drou were sentenc...
8,The Financial Conduct Authority (FCA) said tha...,"Investors must be quoted an ""all-in fee"" to ma...",The Financial Conduct Authority (FCA) has anno...
9,Yonhap news agency quoted a South Korean offic...,North Korean leader Kim Jong-il is paying his ...,South Korean officials have reported that Kim ...


In [11]:
df.to_pickle("../generated_results/phi_3_5_mini_results.pkl")

In [12]:
df = pd.read_pickle("../generated_results/phi_3_5_mini_results.pkl")
df

,Article,Human Summary,Generated Summary
0,"The Newport man faces other charges, including...",A 22-year-old man has been charged with causin...,A man in Newport faces additional charges of v...
1,"Staff at RSPCA Gonsal Farm animal centre, in D...",A Shropshire charity has designated October 'B...,The RSPCA Gal Farm animal center in Dorrington...
2,"According to CNN, the former FBI director and ...",Now that Hurricane Junior has blown through Wa...,The FBI and former investigators were caught o...
3,Former leader Nick Paget-Brown resigned on 30 ...,The new leader of Kensington and Chelsea Counc...,Former leader Nick Paget-B resigned after crit...
4,That makes it more serious than a technical co...,"The index of the UK's biggest 100 companies, t...",The text discusses the current market situatio...
5,Gorse fires have been big news this week and t...,"A ""river of filth"", a spate of gorse fires, an...",Recent fires have been causing significant new...
6,"David Davies, Ian Lucas, Albert Owen and Gerai...",Four Welsh MPs are standing for election as ch...,"David Davies, Ian, Owen, and Geraint Davies ar..."
7,Three auctioneers at Hotel Drouot also receive...,A French court has jailed 35 porters at the co...,Three hotel porters at Hotel Drou were sentenc...
8,The Financial Conduct Authority (FCA) said tha...,"Investors must be quoted an ""all-in fee"" to ma...",The Financial Conduct Authority (FCA) has anno...
9,Yonhap news agency quoted a South Korean offic...,North Korean leader Kim Jong-il is paying his ...,South Korean officials have reported that Kim ...


### METEOR (Metric for Evaluation of Translation with Explicit Ordering)

In [13]:
meteor = evaluate.load("meteor")

peft_model_meteor_results = meteor.compute(
    predictions = generated_summaries,
    references = human_summaries[0:len(generated_summaries)]
)

print(peft_model_meteor_results)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.05213202795659032}


### ROUGE-N (Recall-Oriented Understudy for Gisting Evaluation)

In [14]:
rouge = evaluate.load("rouge")

peft_model_rouge_results = rouge.compute(
    predictions = generated_summaries,
    references = human_summaries[0:len(generated_summaries)],
    use_aggregator = True,
    use_stemmer = True
)

print(peft_model_rouge_results)

{'rouge1': 0.02607380871041317, 'rouge2': 0.005834941016358478, 'rougeL': 0.019899297300611428, 'rougeLsum': 0.021006378128991143}


### BERTScore

In [15]:
from statistics import mean

bert_score = evaluate.load("bertscore")

peft_model_bert_score_results = bert_score.compute(
    predictions = df['Generated Summary'],
    references = df['Human Summary'][0:len(df['Generated Summary'])],
    lang = "en"
)

print(mean(peft_model_bert_score_results['precision']))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.7674983775615692


### BLEU (BiLingual Evaluation Understudy)

In [16]:
bleu_score = evaluate.load("bleu")

peft_model_bleu_score_results = bleu_score.compute(
    predictions = generated_summaries,
    references = human_summaries[0:len(generated_summaries)]
)

print(peft_model_bleu_score_results)

{'bleu': 0.0007161077720742173, 'precisions': [0.005706833247495473, 0.001346275304989529, 0.00037412081608220683, 9.1489786412935e-05], 'brevity_penalty': 1.0, 'length_ratio': 97.71266233766234, 'translation_length': 120382, 'reference_length': 1232}


### Average Inference Time

In [ ]:
import time

inference_times = []

for idx, article in enumerate(articles):
    input_ids = tokenizer(prompt, return_tensors='pt')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids.to(device)

    FastLanguageModel.for_inference(peft_model)
    inference_start_time = time.time()
    model_output = peft_model.generate(**input_ids, max_new_tokens = 6000, temperature = 0.1)
    prompt_length = input_ids['input_ids'].shape[1]
    model_text_output = tokenizer.decode(model_output[0][prompt_length:], skip_special_tokens = True)
    inference_end_time = time.time()
    inference_time = inference_end_time - inference_start_time
    inference_times.append(inference_time)

mean_inference_time = mean(inference_times)
print(f"Average Inference Time: {mean_inference_time}")